In [47]:
import mysql.connector as sql
import pandas as pd
import datetime as dt
import numpy as np
from sorted_months_weekdays import *
from sort_dataframeby_monthorweek import *
import matplotlib.pyplot as plt
import math
import requests
import json
from pandas.io.json import json_normalize
import matplotlib.dates as mdates
from datetime import date, timedelta
from requests.packages.urllib3.util.retry import Retry

In [2]:
db_connection = sql.connect(host='localhost', database='collection_users', user='pandas', password='password')
sbItems = pd.read_sql('SELECT * FROM UsersShoeboxItems', con=db_connection)
sbItems['tool_id'] = sbItems['tool_id'].astype('category')
sbItems['extension'].replace(r'^\s*$', np.NaN, regex=True, inplace=True)
sbItems['extension'] = sbItems['extension'].astype('category')
sbItems['mimetype'] = sbItems['mimetype'].astype('category')
sbItems['created'] = pd.to_datetime(sbItems['created'], unit='s')
sbItems['lastmodified'] = pd.to_datetime(sbItems['lastmodified'], unit='s')
sbItems['deleted'].replace(0, np.nan, inplace=True)
sbItems['deleted'] = pd.to_datetime(sbItems['deleted'], unit='s')
# sbItems['refers_to_uid'].replace(0, np.nan, inplace=True)
sbItems['refers_to_uid'] = sbItems['refers_to_uid'].astype(int)
sbItems['fingerprint'].replace(r'^\s*$', np.NaN, regex=True, inplace=True)
sbItems['title'].replace(r'^\s*$', np.NaN, regex=True, inplace=True)
sbItems['description'].replace(r'^\s*$', np.NaN, regex=True, inplace=True)
sbItems['secret'].replace(r'^\s*$', np.NaN, regex=True, inplace=True)
sbItems['secret_o'].replace(r'^\s*$', np.NaN, regex=True, inplace=True)
sbItems['extras'].replace(r'^\s*$', np.NaN, regex=True, inplace=True)

In [12]:
url = 'https://api.collection.cooperhewitt.org/rest/?method=cooperhewitt.objects.getInfo&access_token=729cca8ffeb40cd2e725f5301e034134&object_id=69155331'
resp = requests.request('GET', url)
resp

<Response [200]>

In [13]:
resp.status_code

200

In [17]:
items = json.loads(resp.text)
items = pd.json_normalize(items)
items

,stat,event_publishing_state,object.id,object.tms:id,object.accession_number,object.title,object.title_raw,object.url,object.has_no_known_copyright,object.department_id,...,object.justification,object.gallery_text,object.label_text,object.videos,object.on_display,object.woe:country,object.type,object.images,object.participants,object.is_loan_object
0,ok,ok,69155331,282992,"7.2015.1a,b","Scale Models, Architecture is Everywhere, 2015",Architecture is Everywhere,https://collection.cooperhewitt.org/objects/69...,None,404529577,...,None,Sou Fujimoto Architects created this series of...,None,None,None,None,Scale models,[{'b': {'url': 'https://images.collection.coop...,"[{'person_id': '69154757', 'role_id': '3523665...",1


In [45]:
sbItems_wo_duplicates = sbItems[~sbItems['refers_to_uid'].duplicated()]

In [46]:
sbItems_up16 = sbItems_wo_duplicates[sbItems_wo_duplicates['created'] < '2016-01-01']
# sbItems_up16 = sbItems_up16.loc[100:110]
sbItems_up16

,id,user_id,action_id,tool_id,bundle_id,refers_to_id,refers_to_uid,fingerprint,title,description,...,mimetype,is_public,created,lastmodified,deleted,secret_o,extras,status_id,processed,input_id
0,789443,199,2,0,789383,0,0,e6c8ee7e3c4d75c6292befc26c24dc884db71cd4,NaN,NaN,...,image/png,0,2014-09-30 20:16:24,2014-09-30 20:38:58,NaT,UqbTQrnbo5BV,NaN,0,0,0
2,789969,231,1,0,789967,1,18706081,NaN,NaN,NaN,...,type/prs.hyperlink,0,2014-10-02 19:59:12,2014-10-02 21:43:36,NaT,NaN,NaN,0,0,0
3,789989,231,1,0,789967,1,18771063,NaN,NaN,NaN,...,type/prs.hyperlink,0,2014-10-02 21:24:34,2014-10-02 21:43:36,NaT,NaN,NaN,0,0,0
4,789991,231,1,0,789967,1,18630089,NaN,NaN,NaN,...,type/prs.hyperlink,0,2014-10-02 19:59:41,2014-10-02 21:43:36,NaT,NaN,NaN,0,0,0
5,789993,231,1,0,789967,1,18731743,NaN,NaN,NaN,...,type/prs.hyperlink,0,2014-10-02 19:59:41,2014-10-02 21:43:36,NaT,NaN,NaN,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439776,91903851,8380,2,10,91891785,0,1451594899,9c6acf0aba6739ac924015ac9ca04be5fcc5ba5b,My Building,Made at Cooper Hewitt Museum using a Collectio...,...,application/sla,0,2015-12-31 20:48:19,2016-01-06 18:18:32,NaT,kzyRi627dShR,"{""fingerprints"":{""o"":{""sha1"":""685f6b41e441ddd6...",0,0,3
439869,91908693,8397,2,15,91873709,0,1451598186,567485c2496be079f229639df01685772d25cd74,My Free Form Object,Made at Cooper Hewitt Museum using a Collectio...,...,application/sla,0,2015-12-31 21:43:06,2016-01-06 18:18:32,NaT,M5otUP2EKPbl,"{""fingerprints"":{""o"":{""sha1"":""8f70e33fc4e32d2e...",0,0,3
439912,91908805,8391,2,10,91905535,0,1451598345,917a74bb095c8b150143e9e6a1a3d0d1047c7509,My Building,Made at Cooper Hewitt Museum using a Collectio...,...,application/sla,0,2015-12-31 21:45:45,2016-01-06 18:18:32,NaT,Iq9yUp8MAaK6,"{""fingerprints"":{""o"":{""sha1"":""727258f1af03935f...",0,0,3
1057162,108783009,158,1,0,1513947,1,18666275,NaN,NaN,NaN,...,type/prs.hyperlink,0,2015-02-03 18:05:39,2016-09-18 03:01:12,NaT,NaN,NaN,0,0,1


In [ ]:
sbItems_up16['refers_to_uid'] = sbItems_up16['refers_to_uid'].astype(str)
items_info = None
errs = []
for item in sbItems_up16['refers_to_uid']:
    try:
        url = 'https://api.collection.cooperhewitt.org/rest/?method=cooperhewitt.objects.getInfo&access_token=729cca8ffeb40cd2e725f5301e034134&object_id=' + item
#         resp = requests.request('GET', url)
        resp = fetch_URL(url)
        items = json.loads(resp.text)
        items = pd.json_normalize(items)
        if items_info is None:
            items_info = items
        else:
            items_info = pd.concat([items_info, items])
    except RuntimeError as e:
        errs.append(e)
items_info

/Users/shirin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [52]:
def fetch_URL(url):
    session = requests.Session()
    retry = Retry(connect = 3, backoff_factor = 0.5)
    adapter = requests.adapters.HTTPAdapter(max_retries = retry)
    session.mount('https://', adapter)
    
    return session.get(url)

In [29]:
items_info

,stat,event_publishing_state,error.code,error.error,error.message,object.id,object.tms:id,object.accession_number,object.title,object.title_raw,...,object.participants,object.woe:country_name,object.is_loan_object,object.dimensions_raw.height,object.dimensions_raw.diameter,object.dimensions_raw,object.dimensions_raw.warp,object.dimensions_raw.weft,object.dimensions_raw.pages,object.dimensions_raw.digital file
0,error,ok,404.0,Not a valid object ID,Not a valid object ID,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,error,ok,404.0,Not a valid object ID,Not a valid object ID,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,ok,ok,NaN,NaN,NaN,18706081,228868,1959-56-1-e,"Fork\r\n(e,f) Fork (England)",,...,"[{'person_id': '68736529', 'role_id': '3535153...",United Kingdom,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,ok,ok,NaN,NaN,NaN,18771063,261842,2010-31-311,Sidewall (France),,...,"[{'person_id': '18050217', 'role_id': '3535153...",France,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,ok,ok,NaN,NaN,NaN,18630089,189885,1988-62-206,"Drawing, Textile Design: Monte Zuma, 1910–12",Textile Design: Monte Zuma,...,"[{'person_id': '18539195', 'role_id': '2318792...",Austria,0.0,"[40.60, centimeters]",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,ok,ok,NaN,NaN,NaN,18504289,163048,1980-14-872,Matchsafe,None,...,"[{'person_id': '18539175', 'role_id': '3535153...",NaN,0.0,"[6.20, centimeters]",NaN,NaN,NaN,NaN,NaN,NaN
0,ok,ok,NaN,NaN,NaN,18247707,30699,1921-6-308-2,Print (France),,...,"[{'person_id': '18049621', 'role_id': '3535153...",France,0.0,NaN,NaN,None,NaN,NaN,NaN,NaN
0,ok,ok,NaN,NaN,NaN,18141561,13345,1904-20-175,Mount,None,...,"[{'person_id': '18050451', 'role_id': '3535153...",NaN,0.0,"[2.70, centimeters]","[9.60, centimeters]",NaN,NaN,NaN,NaN,NaN
0,ok,ok,NaN,NaN,NaN,18696123,223638,2002-8-4,"Poster Announcement, Cal Arts Program in Graph...",Cal Arts Program in Graphic Design Announcemen...,...,"[{'person_id': '18041625', 'role_id': '3535153...",NaN,0.0,NaN,NaN,None,NaN,NaN,NaN,NaN


In [37]:
sbItems['refers_to_uid'].iloc[648]

18193437

In [53]:
url = 'https://api.collection.cooperhewitt.org/rest/?method=cooperhewitt.objects.getInfo&access_token=729cca8ffeb40cd2e725f5301e034134&object_id=18193437'
resp = fetch_URL(url)
resp

<Response [200]>

In [54]:
test = json.loads(resp.text)
test = pd.json_normalize(test)
test

,stat,event_publishing_state,object.id,object.tms:id,object.accession_number,object.title,object.title_raw,object.url,object.has_no_known_copyright,object.department_id,...,object.gallery_text,object.label_text,object.videos,object.on_display,object.woe:country,object.type,object.images,object.participants,object.woe:country_name,object.is_loan_object
0,ok,ok,18193437,22345,1917-4-1293,"Drawing, Classical Ruin, Trees, 1844","Classical Ruin, Trees",https://collection.cooperhewitt.org/objects/18...,1,35347493,...,None,None,None,None,23424977,Drawing,[{'b': {'url': 'https://images.collection.coop...,"[{'person_id': '18041941', 'role_id': '3523656...",United States,0


In [ ]:
items_info